# 1. Building a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto

In [1]:
!pip install beautifulsoup4

     |████████████████████████████████| 122kB 16.9MB/s eta 0:00:01


In [2]:
import numpy as np 
import pandas as pd 
import requests 
from bs4 import BeautifulSoup # library to parse HTML and XML document
print("Libraries Imported Correctly")

Libraries Imported Correctly


### Now lets load the data from the WikiPedia page and parse it using beautifulsoup and append it to the postal, borough and neigborhood lists

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')
postalCode = []
borough = []
neighborhood = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text.rstrip('\n'))
        borough.append(cells[1].text.rstrip('\n'))
        neighborhood.append(cells[2].text.rstrip('\n')) 

### And finally lets create the DataFrame, drop the Not assigned rows and group the postal code

In [4]:
toronto_df = pd.DataFrame({"PostalCode": postalCode,
                           "Borough": borough,
                           "Neighborhood": neighborhood})

toronto_df = toronto_df[toronto_df['Borough'] !='Not assigned'].reset_index(drop=True)
#In this part we are grouping our data and then concatenating the strings
toronto_df = toronto_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
toronto_df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
toronto_df.shape

(103, 3)

# 2. Get the geographical coordinates of the neighborhoods in Toronto.

In [6]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')
geospatial_data.columns=['PostalCode','Latitude','Longitude']
geospatial_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Lets make a joind between the two dataframes

In [7]:
toronto_geospatial = pd.merge(toronto_df, geospatial_data, how='inner', on = 'PostalCode')
toronto_geospatial

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# 3. Explore and cluster the neighborhoods in Toronto base on the category for each venue

In [8]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.colors as plt

from sklearn.cluster import KMeans

import folium 

In [9]:
# create map of Toronto using latitude and longitude values
latitude = 43.653963
longitude = -79.387207
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geospatial['Latitude'], toronto_geospatial['Longitude'], toronto_geospatial['Borough'], toronto_geospatial['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [10]:
borough_names = list(toronto_geospatial.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [11]:
toronto_geospatial = toronto_geospatial[toronto_geospatial['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_geospatial.shape)
toronto_geospatial.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [71]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geospatial['Latitude'], toronto_geospatial['Longitude'], toronto_geospatial['Borough'], toronto_geospatial['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [13]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QXIFSECI13O4X31UGS2OSF4H5LZCOE2ZLSFNFIH1DES4WJMH' # your Foursquare ID
CLIENT_SECRET = '40XIM2HQVWNSBHB4NGROROIQJKEE2QIREJVQ0A0R0SJ1FBC4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QXIFSECI13O4X31UGS2OSF4H5LZCOE2ZLSFNFIH1DES4WJMH
CLIENT_SECRET:40XIM2HQVWNSBHB4NGROROIQJKEE2QIREJVQ0A0R0SJ1FBC4


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
radius = 500
LIMIT = 100
latitude = 43.653963
longitude = -79.387207
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(toronto_geospatial['Neighborhood'],toronto_geospatial['Latitude'],toronto_geospatial['Longitude'])
toronto_venues

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
...,...,...,...,...,...,...,...
1618,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,The Ashbridge Estate,43.664691,-79.321805,Garden
1619,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1620,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1621,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio


In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,17,17,17,17,17,17
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


### Lets analyze the data that we get from foursquare, firt lets see how many different categories we get from the venues and then search for mexican restaurants to see if there already a some restaurants in this areas

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print(toronto_venues['Venue Category'].unique()[:100])
print("Mexican Restaurant" in toronto_venues['Venue Category'].unique())

There are 236 uniques categories.
['Trail' 'Health Food Store' 'Pub' 'Neighborhood' 'Cosmetics Shop'
 'Greek Restaurant' 'Ice Cream Shop' 'Italian Restaurant' 'Brewery'
 'Fruit & Vegetable Store' 'Yoga Studio' 'Pizza Place' 'Restaurant'
 'Juice Bar' 'Bookstore' 'Bubble Tea Shop' 'Dessert Shop'
 'Furniture / Home Store' 'Grocery Store' 'Spa' 'Coffee Shop' 'Bakery'
 'Caribbean Restaurant' 'Indian Restaurant' 'Japanese Restaurant' 'Café'
 'Lounge' 'Frozen Yogurt Shop' 'Liquor Store' 'American Restaurant' 'Gym'
 'Fish & Chips Shop' 'Fast Food Restaurant' 'Sushi Restaurant' 'Park'
 'Pet Store' 'Steakhouse' 'Burrito Place' 'Movie Theater' 'Sandwich Place'
 'Food & Drink Shop' 'Fish Market' 'Gay Bar' 'Thai Restaurant'
 'Seafood Restaurant' 'Cheese Shop' 'Comfort Food Restaurant'
 'Middle Eastern Restaurant' 'Stationery Store' 'Coworking Space'
 'Wine Bar' 'Latin American Restaurant' 'Gastropub' 'Gym / Fitness Center'
 'Bar' 'Convenience Store' 'Bank' 'Diner' 'Clothing Store' 'Swim School'
 'B

### As we can see theres Mexican restaurants in the categories of the venue
### For the next part lets analise each neighborhood to see how many of the venues exists there, for this part we will group the neighborhoods and get the mean of each category to se how many of each venue are in every neighborhood

In [35]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1623, 237)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Once this was done lets filter just the mexican restaurants

In [56]:
#to_grouped = to_onehot.groupby(["Neighborhoods"]).count().reset_index()
to_onehot[["Neighborhoods",'Mexican Restaurant']][to_onehot['Mexican Restaurant'] > 0]
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

In [82]:
mexican = to_grouped[["Neighborhoods","Mexican Restaurant"]]
mexican[mexican['Mexican Restaurant']  > 0]

,Neighborhoods,Mexican Restaurant
6,Church and Wellesley,0.012346
13,"Garden District, Ryerson",0.010000
15,"High Park, The Junction South",0.086957
17,"Kensington Market, Chinatown, Grange Park",0.049180
21,"North Toronto West, Lawrence Park",0.052632
23,"Queen's Park, Ontario Provincial Government",0.030303


In [61]:

# set number of clusters
toclusters = 3

to_clustering = mexican.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [63]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = mexican.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Mexican Restaurant,Cluster Labels
0,Berczy Park,0.0,0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0
2,"Business reply mail Processing Centre, South C...",0.0,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0
4,Central Bay Street,0.0,0


In [64]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1623, 9)


,Neighborhood,Mexican Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [65]:
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1623, 9)


,Neighborhood,Mexican Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.000000,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
28,"Runnymede, Swansea",0.000000,0,43.651571,-79.484450,A Dark Horse,43.649533,-79.483056,Bar
28,"Runnymede, Swansea",0.000000,0,43.651571,-79.484450,Tim Hortons,43.648526,-79.485066,Coffee Shop
28,"Runnymede, Swansea",0.000000,0,43.651571,-79.484450,Heart,43.650310,-79.480125,Dessert Shop
28,"Runnymede, Swansea",0.000000,0,43.651571,-79.484450,Java Joe's,43.650424,-79.479755,Café
...,...,...,...,...,...,...,...,...,...
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,nodo,43.665303,-79.465621,Italian Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Lithuania Park,43.658667,-79.463038,Park
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Silk,43.665291,-79.466238,Thai Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,The Beet Organic Café,43.665340,-79.467137,Café


In [75]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(lat, lon, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [76]:
map_clusters.save('map_clusters.html')

In [87]:
#Cluster 0
last = to_merged.loc[to_merged['Cluster Labels'] == 0]
last['Neighborhood'].unique().tolist()

['Berczy Park',
 'Runnymede, Swansea',
 'St. James Town',
 'Richmond, Adelaide, King',
 'Roselawn',
 'Rosedale',
 'St. James Town, Cabbagetown',
 'Little Portugal, Trinity',
 'Moore Park, Summerhill East',
 'Regent Park, Harbourfront',
 'Parkdale, Roncesvalles',
 'University of Toronto, Harbord',
 'India Bazaar, The Beaches West',
 'Lawrence Park',
 'Toronto Dominion Centre, Design Exchange',
 'The Danforth West, Riverdale',
 'The Beaches',
 'The Annex, North Midtown, Yorkville',
 'Stn A PO Boxes',
 'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park',
 'Studio District',
 'First Canadian Place, Underground city',
 'Davisville',
 'Commerce Court, Victoria Hotel',
 'Davisville North',
 'Dufferin, Dovercourt Village',
 'Central Bay Street',
 'Church and Wellesley',
 'Christie',
 'Harbourfront East, Union Station, Toronto Islands',
 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport',
 'Business reply mail Processing 

In [78]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Mexican Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
17,"Kensington Market, Chinatown, Grange Park",0.049180,1,43.653206,-79.400049,Icha Tea 一茶間,43.651336,-79.397370,Tea Room
23,"Queen's Park, Ontario Provincial Government",0.030303,1,43.662301,-79.389494,Hart House Music Room,43.663758,-79.395027,Music Venue
17,"Kensington Market, Chinatown, Grange Park",0.049180,1,43.653206,-79.400049,Poetry Jazz Cafe,43.654975,-79.402371,Jazz Club
17,"Kensington Market, Chinatown, Grange Park",0.049180,1,43.653206,-79.400049,Sonic Boom,43.650859,-79.396985,Record Shop
17,"Kensington Market, Chinatown, Grange Park",0.049180,1,43.653206,-79.400049,Maker Pizza,43.650401,-79.398040,Pizza Place
...,...,...,...,...,...,...,...,...,...
21,"North Toronto West, Lawrence Park",0.052632,1,43.715383,-79.405678,The Captain’s Boil,43.715789,-79.400187,Seafood Restaurant
21,"North Toronto West, Lawrence Park",0.052632,1,43.715383,-79.405678,St. Clements - Yonge Parkette,43.712062,-79.404255,Park
21,"North Toronto West, Lawrence Park",0.052632,1,43.715383,-79.405678,Second Cup,43.714583,-79.400120,Café
21,"North Toronto West, Lawrence Park",0.052632,1,43.715383,-79.405678,GAP,43.715450,-79.400089,Clothing Store


In [79]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Mexican Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Tim & Sue's No Frills,43.664243,-79.468643,Grocery Store
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,ROUX,43.665418,-79.462392,Cajun / Creole Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Junction Grill,43.665210,-79.468461,Diner
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Pascal's Baguette & Bagels,43.665426,-79.466176,Bakery
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Chica’s Nashville Hot Chicken,43.665295,-79.464888,Fried Chicken Joint
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,A&W,43.665520,-79.464731,Fast Food Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,La Revolucion,43.665460,-79.464308,Mexican Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Pandemonium,43.665533,-79.466931,Bookstore
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Isaan Der,43.665311,-79.468078,Thai Restaurant
15,"High Park, The Junction South",0.086957,2,43.661608,-79.464763,Playa Cabana Cantina,43.665315,-79.465548,Mexican Restaurant
